In [102]:
import numpy as np
from typing import Tuple, Union

# Iterative Techniques in Matrix Algebra

## Chapter 7.3 The Jacobi and Gauss-Siedel Iterative Techniques

### Jacobi Method
$$x^{(k)}_i=\sum^n_{\substack{j\neq i \\ j=1}}\biggl(-\frac{a_{ij}x^{(k)}_{j}}{a_{ii}}\biggr)+\frac{b_i}{a_{ii}}$$

In [103]:
def Jacobi(A:np.ndarray, x:np.ndarray, b:np.ndarray, iter:int, tol:float=1e-5) -> np.ndarray:
    x_ = np.copy(x)
    for iter in range(iter):
        prev_x = np.copy(x_)
        for idx in range(A.shape[0]):
            mask = np.arange(len(x_)) != idx
            x_[idx] = 1 / (A[idx][idx]) * (b[idx] - np.sum(A[idx][mask] @ prev_x[mask]))
        # if np.linalg.norm(x_ - prev_x) < tol:
        #     print("Iteration stopped, norm difference < tolerance.")
        #     break
    return x_

### Gauss-Siedel Method
$$x^{(k)}_i=\frac{1}{a_{ii}}\biggl[
    -\sum^{i-1}_{j=1}(a_{ij}x^{(k)}_j)-\sum^{n}_{j=i+1}(a_{ij}x^{(k-1)}_j)+b_i
\biggr]$$


In [104]:
def GaussSeidel(A:np.ndarray, x:np.ndarray, b:np.ndarray, iter:int, tol:float=1e-5) -> np.ndarray:
    x_ = np.copy(x)
    for iter in range(iter):
        prev_x = np.copy(x_)
        for idx in range(A.shape[0]):
            mask = np.arange(len(x_)) != idx
            x_[idx] = 1 / (A[idx][idx]) * (b[idx] - np.sum(A[idx][mask] @ x_[mask]))
        # if np.linalg.norm(x_ - prev_x) < tol:
        #     print("Iteration stopped, norm difference < tolerance.")
        #     break
    return x_

## Chapter 7.4 Relaxation Techniques for Solving Linear Systems

### Sucessive Over-Relaxation Method
$$x^{(k)}_i=(1-\omega)x^{(k-1)}_i+\frac{\omega}{a_{ii}}\biggl[
    -\sum^{i-1}_{j=1}(a_{ij}x^{(k)}_j)-\sum^{n}_{j=i+1}(a_{ij}x^{(k-1)}_j)+b_i
\biggr]$$
where $\omega > 1$

In [105]:
def SOR(A:np.ndarray, x:np.ndarray, b:np.ndarray, iter:int, omega=1.5, tol:float=1e-5) -> np.ndarray:
    x_ = np.copy(x)
    for iter in range(iter):
        prev_x = np.copy(x_)
        for idx in range(A.shape[0]):
            mask = np.arange(len(x_)) != idx
            x_[idx] = (1 - omega) * prev_x[idx] + (omega / (A[idx][idx]) * (b[idx] - np.sum(A[idx][mask] @ x_[mask])))
        # if np.linalg.norm(x_ - prev_x) < tol:
        #     print("Iteration stopped, norm difference < tolerance.")
        #     break
    return x_

## Chapter 7.6 The Conjugate Gradient Method

In [106]:
def dot(alpha, beta):
    return np.dot(alpha.flatten(), beta.flatten())

# def ConjugateGradient(A:np.ndarray, x:np.ndarray, b:np.ndarray, v:np.ndarray, tol=1e-6) -> np.ndarray:
#     x_ = x.copy().astype(float)
#     for k in range(v.shape[0]):
#         t = np.dot(v[:, k], b - A @ x_) / (np.dot(v[:, k], A @ v[:, k]))
#         x_ += (t * v[:, k]).reshape(x_.shape)
#         if np.linalg.norm(b - A @ x_) < tol:
#             return x_
#     return x_

def ConjugateGradient(A:np.ndarray, x:np.ndarray, b:np.ndarray, n:int=None, tol=1e-6) -> np.ndarray:
    x_ = x.copy().astype(float)
    r = b - A @ x_
    v = r.copy()

    if n is None:
        n = int(len(b) * 10)

    for _ in range(n):
        if np.linalg.norm(r) < tol:
            return x_
        s = 1 / dot(r, r)
        t = dot(r, r) / dot(v, A @ v)
        x_ += (t * v).reshape(x_.shape)
        r -= t * A @ v
        s *= dot(r, r)
        v = r + s * v

    return x_

def PreconditionConjugateGradient(A:np.ndarray, C:np.ndarray, x:np.ndarray, b:np.ndarray, n:int=None, tol=1e-6) -> np.ndarray:
    x_ = x.copy().astype(float)
    r = b - A @ x_
    w = C @ r
    v = C.T @ w
    if n is None:
        n = int(len(b) * 10)

    for _ in range(n):
        if np.linalg.norm(v) < tol:
            return x_
        s = 1 / dot(w, w)
        t = dot(w, w) / dot(v, A @ v)
        x_ += (t * v).reshape(x_.shape)
        r -= t * A @ v
        if np.linalg.norm(r) < tol:
            return x_
        w = C @ r
        s *= dot(w, w)
        v = C.T @ w + s * v
    return x_

In [107]:
def BiconjugateGradient(A:np.ndarray, x0:np.ndarray, b:np.ndarray, n:int=None, tol:float=1e-6) -> np.ndarray:
    x = x0.copy()
    r = b - A @ x
    r_ = r.copy().flatten()
    v, v_ = r.copy(), r_.copy()
    if n is None:
        n = int(len(b) * 10)
    for _ in range(n):
        if np.linalg.norm(r) < tol:
            return x
        s = 1 / dot(r_, r)
        t = dot(r_, r) / dot(v_, A @ v)
        x += (t * v).reshape(x.shape)
        r -= t * A @ v
        r_ -= t * (A.T @ v_)
        s *= dot(r_, r)
        v = r + s * v
        v_ = r_ + s.conj() * v_
    return x

def PreconditionBiconjugateGradient(A:np.ndarray, C:np.ndarray, x0:np.ndarray, b:np.ndarray, n:int=None, tol=1e-6) -> np.ndarray:
    x = x0.copy()
    r = b - A @ x
    r_ = r.copy().flatten()
    v, v_ = C @ r.copy(), C.T @ r_.copy()
    if n is None:
        n = int(len(b) * 10)
    for _ in range(n):
        if np.linalg.norm(r) < tol:
            return x
        s = 1 / dot(r_, C @ r)
        t = dot(r_, C @ r) / dot(v_, A @ v)
        x += (t * v).reshape(x.shape)
        r -= t * A @ v
        r_ -= t * (A.T @ v_)
        s *= dot(r_, C @ r)
        v = C @ r + s * v
        v_ = (C.T @ r_).flatten() + s.conj() * v_
    return x

In [108]:
def BiconjugateGradientSTAB(A:np.ndarray, x0:np.ndarray, b:np.ndarray, n:int=None, tol:float=1e-6) -> np.ndarray:
    x = x0.copy()
    r = b - A @ x
    r_ = r.copy().flatten()
    rho = dot(r_, r)
    p = r.copy()
    if n is None:
        n = int(len(b) * 10)
    for _ in range(n):
        if np.linalg.norm(r) < tol:
            return x
        v = A @ p
        alpha = rho / dot(r_, v)
        r -= alpha * v
        s = r.copy()
        t = A @ s
        omega = dot(t, s) / dot(t, t)
        r -= omega * t
        x += alpha * p + omega * s
        beta = alpha / (omega * rho)
        rho = dot(r_, r)
        beta *= rho
        p = r + beta * (p - omega * v)
    return x

def PreconditionBiconjugateGradientSTAB(A:np.ndarray, C:np.ndarray, x0:np.ndarray, b:np.ndarray, n:int=None, tol:float=1e-6) -> np.ndarray:
    x = x0.copy()
    r = b - A @ x
    r_ = r.copy().flatten()
    rho = dot(r_, r)
    p = r.copy()
    if n is None:
        n = int(len(b) * 10)
    for _ in range(n):
        if np.linalg.norm(r) < tol:
            return x
        v = A @ C @ p
        alpha = rho / dot(r_, v)
        r -= alpha * v
        s = r.copy()
        t = A @ C @ s
        omega = dot(t, s) / dot(t, t)
        r -= omega * t
        x += alpha * C @ p + omega * C @ s
        beta = alpha / (omega * rho)
        rho = dot(r_, r)
        beta *= rho
        p = r + beta * (p - omega * v)
    return x

In [109]:
def MINRES(A:np.ndarray, x:np.ndarray, b:np.ndarray, tol=1e-6) -> np.ndarray:
    x_ = x.copy().astype(float)
    r = b - A @ x_
    v0 = r.copy()
    s0 = A @ v0
    v1 = v0.copy()
    s1 = s0.copy()
    for k in range(x.shape[0]):
        v2 = v1.copy()
        v1 = v0.copy()
        s2 = s1.copy()
        s1 = s0.copy()
        t = dot(r, s1) / dot(s1, s1)
        x_ += (t * v1).reshape(x_.shape)
        r -= t * s1
        if np.linalg.norm(r) < tol:
            return x_
        s0 = A @ s1
        beta = dot(s0, s1) / dot(s1, s1)
        v0 = s1 - beta * v1
        s0 -= beta * s1
        if k:
            beta = dot(s0, s2) / dot(s2, s2)
            v0 -= beta * v2
            s0 -= beta * s2
    return x_

In [112]:
np.set_printoptions(10)

A = np.random.rand(3, 3)

b = np.array([
    [2],
    [4],
    [-1],
])

x = np.array([
    [0.],
    [0.],
    [0.],
], dtype=np.float32)



print("Biconjugate Gradient")
print(BiconjugateGradient(A, x, b))

print("Preconditioned Biconjugate Gradient")
print(PreconditionBiconjugateGradient(A, np.eye(A.shape[0]), x, b))

print("Biconjugate Gradient Stabilized")
print(BiconjugateGradientSTAB(A, x, b))

print("Preconditioned Biconjugate Gradient")
print(PreconditionBiconjugateGradientSTAB(A, np.eye(A.shape[0]), x, b))

print("MINRES")
print(MINRES(A, x, b))

# print("Preconditioned Conjugate Gradient")
# print(PreconditionConjugateGradient(A, 1 / np.sqrt(np.diag(A)) * np.eye(A.shape[0]), x, b, 0.01))

Biconjugate Gradient
[[ 9.084332 ]
 [-0.9101125]
 [-8.020199 ]]
Preconditioned Biconjugate Gradient
[[ 9.084332 ]
 [-0.9101125]
 [-8.020199 ]]
Biconjugate Gradient Stabilized
[[ 9.084332 ]
 [-0.9101124]
 [-8.020199 ]]
Preconditioned Biconjugate Gradient
[[ 9.084332 ]
 [-0.9101124]
 [-8.020199 ]]
MINRES
[[ 9.0843320244]
 [-0.9101124179]
 [-8.0201983357]]


In [113]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import bicg, cg, minres, bicgstab
A_ = A.copy()
A = csc_matrix(A)
b = np.array([2., 4., -1.])
x, exitCode = bicgstab(A, b, atol=1e-6, rtol=1e-6)
print(x)
x, exitCode = bicg(A, b, atol=1e-6, rtol=1e-6)
print(x)
# np.allclose(A.dot(x), b)

[ 9.0843320244 -0.9101124179 -8.0201983357]
[ 9.0843320244 -0.9101124179 -8.0201983357]


# Appendix
## Theorem 1
